# 4.  スマートフォンの伝送データを画像ファイルとして保存してみよう

このチュートリアルでは、iOSアプリケーション **Visual M2M Motion**  （以下、VM2M Motion とします）からストリーミングされた動画データを画像として保存します。このケースでは、以下の方法を中心に解説します。

* サーバー側に保存されたM-JPEGデータをバイナリデータとして取得する
* 画像ファイル(.jpg)として保存する

## 4.0 事前準備
本シナリオを実施する前に、以下を用意する必要があります。

- 計測用のエッジ
-  VM2M Motionアプリでアップロードした計測（画像を含む）


### 使用データ
本シナリオでは、事前に以下のデータをサーバー側に準備する必要があります。  

|データ項目|本シナリオで登場するデータ名|
|:---|:---|
|時系列データを登録するエッジ|edge1|
|計測情報 (※)|計測名は登場なし|

(※) 使用する時系列データは **intdashチュートリアル2A：スマートフォンの伝送データをリアルタイムに可視化してみよう** と同様に、 VM2M Motion を使用して時系列データを登録します。

### 計測データの作成と時系列データのアップロード
**Visual M2M Motion** を使用してデータをアップロードします。 計測を登録後、計測が新しく生成されたことを **Data Visualizer** で確認してください。

アップロード方法については、 **intdashチュートリアル2A：スマートフォンの伝送データをリアルタイムに可視化してみよう** を参照してください。

Data Visualizerの[Stored Data]に計測が表示されていれば完了です。

<img src="https://user-images.githubusercontent.com/70192465/94385045-103f7280-017f-11eb-8f12-30df59079069.png">

### パッケージのimportとクライアントの生成
`intdash.Client` に与える `url` は intdashサーバーの環境情報から、`username` と `password` はログイン用エッジで発行したアクセス情報を指定してください。  

In [1]:
import pandas as pd

import intdash
from intdash import timeutils

# Create client
client = intdash.Client(
    url = "https://example.intdash.jp",
    username = "edge1",
    password="password_here"
)

これで、事前準備は終了です。

## 4.1 計測に使用したエッジを取得する
VM2M Motionでログインした際に使用したエッジを取得します。

In [2]:
edges = client.edges.list(name='edge1')
edge1 = edges[0]

In [3]:
edge1.name

'edge1'

## 4.2 時系列データを取得する
本シナリオでは、時系列データの取得には `client.data_points` エンドポイントを使用します。信号定義からデータを取得する場合、通常は ``labels`` を指定しますが、本ケースでは信号定義を使用しないため、 `id_queries` で取得します。   
`start` および `end` は前述の手順で作成した計測を含む時刻に変更してください。

※通常は ``data_id`` を指定しますが、JPEGデータの場合は ``data_id`` が固定されているため指定する必要はありません。詳細は、ドキュメント記載の [intdash.data.Jpeg](https://docs.intdash.jp/sdk/python/latest/ja/data.html#intdash.data.JPEG)を確認してください。

In [4]:
dps = client.data_points.list(
    edge_name='edge1',
    start=timeutils.str2timestamp('2020-07-17 00:00:00+09:00'), # change appropriately.
    end=timeutils.str2timestamp('2020-07-18 00:00:00+09:00'),  # change appropriately.
    id_queries=[
        intdash.IdQuery(
            data_type=intdash.DataType.jpeg,
            channel=1
        )
    ]
)

## 4.3 取得したデータを画像ファイルとして保存する
取得したバイナリデータを、そのままJPEGとして保存します。
ここではサンプルとして、取得した時系列データのうち、最初のデータポイントのみを保存します。

In [5]:
data = dps[0] #  Get the first data.

with open(f'./{pd.Timestamp(data.time).value}.jpg', 'wb') as fout:
    # Use the timestamp as the filename.
    fout.write(data.data_payload)

すべてのデータポイントの画像を保存したい場合は、上記を繰り返します。

In [6]:
import os 

save_dir='./images'

for d in dps:
    
    if not os.path.isdir(save_dir):
            os.makedirs(save_dir)
            
    # Use the timestamp as the filename.
    with open(f'{save_dir}/{pd.Timestamp(d.time).value}.jpg', 'wb') as fout:
        fout.write(d.data_payload)